In [29]:
%load_ext autoreload
%autoreload 2
%aimport - pulp
%aimport - pulp.pulp


import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Third-party imports
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from index_travel_accessibility.travel_time_and_centroid import (
    map_predicted_and_existing_hospitals,
)

# Local imports
from ai_planner.planner import (
    HospitalPlanner,
    CRS
)

from index_travel_accessibility.travel_time_and_centroid import (
    get_hospital_df,
)
from main import (
    Index,
    assemble_indexes,
    centroid,
    current_hospital_demand,
    equity_index,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create Equity Index

In [30]:
index_df = assemble_indexes()
# Set the weights for the indexes
weight = {
    Index.FORECAST_DEMAND: 0.25,
    Index.ELDERLY_SHARE: 0.25,
    Index.GISD: 0.25,
    Index.HOSPITAL_CAPACITY: 0.25,
    Index.TRAVEL_TIME: 0.25,
    Index.ACCESSIBILITY: 0.25,
}
equity_df = equity_index(index_df, weight)

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


ARIMA(3,0,0) AIC=-135.65
Current best order (3,0,0)
ARIMA(1,1,0) AIC=429.95
Current best order (1,1,0)


c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-p

ARIMA(1,1,0) AIC=432.50
Current best order (1,1,0)
ARIMA(1,1,0) AIC=452.99
Current best order (1,1,0)
ARIMA(1,1,0) AIC=439.17
Current best order (1,1,0)
ARIMA(1,1,0) AIC=419.81
Current best order (1,1,0)
ARIMA(1,1,0) AIC=467.72
Current best order (1,1,0)


c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\sit

[GISD] Wrote 6 Saarland districts to c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_gisd\data\processed\gisd_norm_saarland.csv


c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


********************************************************************************
Estimated time to run the pipeline ---->  2-3 mins
********************************************************************************
✅ Saved Saarland Hospital Dataset
********************************************************************************
✅ Saved Hospital Coordinates Dataset
********************************************************************************
✅ Saved 5 evenly spread sample points per district
********************************************************************************
✅ Found nearest hospitals to all sample points.
Error at row 19: HTTPConnectionPool(host='router.project-osrm.org', port=80): Max retries exceeded with url: /route/v1/driving/6.914046861333361,49.45416103233337;6.88979,49.4190916?overview=false (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000016724682CF0>, 'Connection to router.project-osrm.org timed out. (connect timeout=None)'))
Erro

### Create Centroids Dataframe

#### Create 1 Centroid per Dataframe

In [31]:

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'index': 'district_code'})


c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[1:] = col.iloc[1:].map(lambda x: int(x.split('/')[0]))
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_travel_accessibility\travel_time_and_centroid.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely

#### Create 5 Centroids per District

In [32]:
# Load the Excel file
file_path = "./../index_travel_accessibility/data/processed/saarland_districts_sample_points.xlsx"
df_sample_points = pd.read_excel(file_path)
SAARLAND_DISTRICT_MAPPING = {
    "Saarlouis": 10044,
    "St. Wendel": 10046,
    "Saarpfalz-Kreis": 10045,
    "Merzig-Wadern": 10042,
    "Neunkirchen": 10043,
    "Regionalverband Saarbrücken": 10041 
}
# Display basic information about the dataframe
df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
df_sample_points = df_sample_points.set_index('district')

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
districts_df = df_sample_points.join(districts_df[['demand', 'equity_index']], how='left')
districts_df["centroid"] = districts_df.apply(
    lambda row: Point(row["longitude"], row["latitude"]),
    axis=1
)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'district': 'district_code'})


C:\Users\ACER\AppData\Local\Temp\ipykernel_10644\384420393.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



### Executing Planner

In [33]:
#STEP1: Generate Candidates and Initialize Graph
n_samples_per_centroid = 5
radius_km = 5
planner = HospitalPlanner(districts_gdf=districts_gdf)
planner.generate_candidates(n_samples_per_centroid=n_samples_per_centroid, radius_km=radius_km)
# planner.init_graph()
# Save the graph file first to avoid re-downloading the graph
# G = ox.graph_from_place("Saarland, Germany", network_type='drive')
# ox.save_graphml(G, "saarland_graph.graphml")
planner.init_graph(graph_path="./../saarland_graph.graphml")

#### Baseline Model
`demand_weight` param to define the weight of current demand in the objective function 
`equity_weight` param to define the weight of the equity index in the objective function
`travel_weight` param to define the weight of travel time in the objection function
`time_threshold` param to define that only candidate points are assigned to a centroid if its travel time is less or equal to the value.


In [ ]:
# #STEP2: Initialize Baseline Model
# planner.init_baseline_model(
#     demand_weight=1,
#     equity_weight=1,
#     travel_weight=1,
#     beds_weight=1,
#     time_threshold=30, #in minutes
#     max_beds_per_hospital=300,
#     min_beds_per_hospital=20,
#     max_beds=1500
# )


#### Add Existing Hospital Constraint
Soft constraint that adds penalty to predicted hospitals if they are located near existing hospital 
`supply_weight` param to define the weight for the supply in the objective function
`distance_threshold` param to define the maximum amount of distance for a candidate to be a neighboor of existing hospitals.

In [ ]:
# # Get hospital data and convert to GeoDataFrame
# hospital_df = get_hospital_df()

# # Convert to GeoDataFrame using latitude and longitude columns
# hospital_gdf = gpd.GeoDataFrame(
#     hospital_df, 
#     geometry=gpd.points_from_xy(hospital_df.Lon, hospital_df.Lat),
#     crs=CRS  # WGS84 coordinate system
# )
# planner.init_baseline_model(
#     demand_weight=0.5,
#     equity_weight=0.5,
#     travel_weight=0.5,
#     beds_weight=0.5,
#     time_threshold=30, #in minutes
#     max_beds_per_hospital=300,
#     min_beds_per_hospital=20,
#     max_beds=1500
# )
# planner.add_existing_hospitals_constraints(
#     gdf=hospital_gdf,
#     supply_weight=0.5,
#     distance_threshold=5000, #in meters
#     num_neighbors=1
# )

In [34]:
# Get hospital data and convert to GeoDataFrame
hospital_df = get_hospital_df()

# Convert to GeoDataFrame using latitude and longitude columns
hospital_gdf = gpd.GeoDataFrame(
    hospital_df, 
    geometry=gpd.points_from_xy(hospital_df.Lon, hospital_df.Lat),
    crs=CRS  # WGS84 coordinate system
)
planner.init_baseline_model(
    # demand_weight=1, # this is added in the Equity Index so no need to set the weight 
    equity_weight=0.4,
    travel_weight=0.4,
    beds_weight=0.3,
    time_threshold=30, #in minutes
    max_beds_per_hospital=300,
    min_beds_per_hospital=20,
    max_beds=1500
)
planner.add_existing_hospitals_constraints(
    gdf=hospital_gdf,
    supply_weight=1,
    distance_threshold=5000, #in meters
    num_neighbors=2
)
planner.add_neighbor_constraints(
    distance_threshold=5000 #in meters
)   

#### Set the number of prediction

In [35]:
#STEP3: Set Number of Predictions
planner.set_num_predictions(k=10)

#### Predict and Plot

In [36]:
selected, assigns, beds = planner.predict()
selected_candidates = planner.candidates_gdf.iloc[selected]
selected_candidates['bed_allocation'] = selected_candidates.index.map(lambda id: beds[id])
# Add Lat and Lon columns using assign
# selected_candidates have Lat and Lon of the new predicted hospitals
selected_candidates = selected_candidates.assign(
    Lat=selected_candidates.geometry.y,
    Lon=selected_candidates.geometry.x
)
map_predicted_and_existing_hospitals(
    'results/maps/main.html', 
    selected_candidates
)

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


✅ Map saved with existing and predicted hospitals!


In [37]:
selected_candidates

,geometry,district_code,node,bed_allocation,Lat,Lon
22,POINT (7.06249 49.33363),10041,585337512,300.0,49.333629,7.062490
48,POINT (7.01826 49.38219),10043,315502192,300.0,49.382192,7.018256
49,POINT (6.97838 49.39632),10043,299898054,20.0,49.396323,6.978378
54,POINT (7.00648 49.38459),10043,1519388866,200.0,49.384586,7.006477
55,POINT (7.0148 49.41436),10043,364884548,20.0,49.414356,7.014796
87,POINT (6.86442 49.36203),10044,449686357,300.0,49.362032,6.864420
89,POINT (6.8611 49.32816),10044,2592430218,300.0,49.328156,6.861097
91,POINT (6.95345 49.45713),10044,1555931481,20.0,49.457135,6.953445
120,POINT (7.00444 49.51657),10046,932351498,20.0,49.516567,7.004437
124,POINT (6.98951 49.51467),10046,2533989378,20.0,49.514674,6.989508


In [38]:
# Modifying the existing hospitals DataFrame to Prediction DataFrame format
existing_hospitals = get_hospital_df()

existing_hospitals.rename(columns={
    'MaxBeds': 'bed_allocation',
    'SiteID': 'node'
}, inplace=True)


# Adding Geometry column to existing_hospitals
existing_hospitals['Lat'] = existing_hospitals['Lat'].round(6)
existing_hospitals['Lon'] = existing_hospitals['Lon'].round(6)
existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)

# Getting district codes from the existing hospitals

# Load Saarland NUTS level 3 shapefile
nuts = gpd.read_file('./../index_travel_accessibility/data/raw/NUTS_RG_01M_2021_4326_LEVL_3.shp')
nuts = nuts.to_crs('EPSG:4326')


# Create geometry column from Lat/Lon
# existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)
# existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

# Spatial join hospitals with NUTS polygons (only needed columns)
existing_hospitals = gpd.sjoin(
    existing_hospitals,
    nuts[['geometry', 'NUTS_NAME']],
    how='left',
    predicate='within'
)

# Map NUTS_NAME to your official Saarland district codes
name_to_code = {
    "Regionalverband Saarbrücken": "10041",
    "Merzig-Wadern": "10042",
    "Neunkirchen": "10043",
    "Saarlouis": "10044",
    "Saarpfalz-Kreis": "10045",
    "St. Wendel": "10046"
}

# Assign district_code based on district_name
existing_hospitals['district_code'] = existing_hospitals['NUTS_NAME'].map(name_to_code)

# Rename NUTS_NAME to district_name
existing_hospitals.rename(columns={'NUTS_NAME': 'district_name'}, inplace=True)

# Select and reorder columns as needed
existing_hospitals = existing_hospitals[['geometry', 'district_code', 'node', 'bed_allocation', 'Lat', 'Lon']]


existing_hospitals.head()



,geometry,district_code,node,bed_allocation,Lat,Lon
0,POINT (7.04132 49.27707),10041,H1,157.0,49.277066,7.041318
1,POINT (6.95997 49.24756),10041,H2,441.0,49.247555,6.959970
2,POINT (6.99259 49.23708),10041,H3,2.0,49.237085,6.992590
3,POINT (6.99414 49.22122),10041,H4,571.0,49.221221,6.994143
4,POINT (6.96355 49.247),10041,H5,277.0,49.246999,6.963554


In [39]:
existing_hospitals['type'] = 'existing'
selected_candidates['type'] = 'predicted'

In [40]:
existing_hospitals.head()
selected_candidates.head()

,geometry,district_code,node,bed_allocation,Lat,Lon,type
22,POINT (7.06249 49.33363),10041,585337512,300.0,49.333629,7.062490,predicted
48,POINT (7.01826 49.38219),10043,315502192,300.0,49.382192,7.018256,predicted
49,POINT (6.97838 49.39632),10043,299898054,20.0,49.396323,6.978378,predicted
54,POINT (7.00648 49.38459),10043,1519388866,200.0,49.384586,7.006477,predicted
55,POINT (7.0148 49.41436),10043,364884548,20.0,49.414356,7.014796,predicted


In [41]:
# If the cols in both exisiting and selected candidates are the same, we can concatenate them
final_results = pd.concat([existing_hospitals, selected_candidates], ignore_index=True)

In [42]:
final_results.to_excel("results/main.xlsx", index=False)